In [8]:
import pyautogui
import cv2
import numpy as np
import easyocr

# Paso 1: Tomar una captura de pantalla de la región izquierda de la pantalla
def take_screenshot(region):
    screenshot = pyautogui.screenshot(region=region)
    screenshot = np.array(screenshot)
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

# Paso 2: Detectar la cuadrícula en la imagen
def detect_grid(image):
    # Convertir la imagen a escala de grises y aplicar un desenfoque gaussiano
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(blurred, 30, 150)

    # Encontrar los contornos en la imagen
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterar sobre los contornos para encontrar la cuadrícula
    for contour in contours:
        # Calcular el perímetro del contorno
        x, y, w, h = cv2.boundingRect(contour)
        
        aspect_ratio = w / float(h)
        if 0.9 <= aspect_ratio <= 1.1 and w > 100 and h > 100:
            # Recortar la cuadrícula detectada
            cropped_image = image[y:y+h, x:x+w]

            # Recortar un 3% adicional de cada borde
            crop_percentage = 0.03
            crop_pixels_y = int(h * crop_percentage)
            crop_pixels_x = int(w * crop_percentage)

            # Aplicar el recorte
            final_cropped_image = cropped_image[crop_pixels_y:h-crop_pixels_y, crop_pixels_x:w-crop_pixels_x]
            
            return final_cropped_image
    
    return None

# Paso 3: Convertir la cuadrícula en una matriz
def grid_to_matrix(grid):
    reader = easyocr.Reader(['en'])
    grid_size = 9
    cell_width = grid.shape[1] // grid_size
    cell_height = grid.shape[0] // grid_size
    matrix = np.zeros((grid_size, grid_size), dtype=int)
    for row in range(grid_size):
        for col in range(grid_size):
            # Calcular las coordenadas de la celda
            x_start = col * cell_width
            y_start = row * cell_height
            x_end = (col + 1) * cell_width
            y_end = (row + 1) * cell_height

            # Recortar la celda y redimensionarla
            cell = grid[y_start:y_end, x_start:x_end]
            resized_cell = cv2.resize(cell, (int(cell.shape[1] * 1.5), (int(cell.shape[0] * 1.5))), interpolation=cv2.INTER_LINEAR)

            # Chequear si la celda es blanca
            mean_color = np.mean(resized_cell)

            if mean_color > 150:  # Si la celda es predominantemente blanca
                matrix[row, col] = 0
            else:  # Si la celda no es blanca
                result = reader.readtext(resized_cell, detail=0, paragraph=False, )
                if result:
                    matrix[row, col] = int(result[0])
                else:
                    matrix[row, col] = -1
    return matrix
    
# Tomar una captura de pantalla de la región izquierda de la pantalla
screen_width, screen_height = pyautogui.size()
region = (0, 0, screen_width // 2, screen_height)
screenshot = take_screenshot(region)

# Detectar la cuadrícula en la imagen
grid = detect_grid(screenshot)

# Convertir la cuadrícula en una matriz
if grid is not None:
    print("Cuadrícula detectada")
    matrix = grid_to_matrix(grid)
    print(matrix)
else:
    print("No se detecto la cuadrícula en la imagen")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1  1  1  1 -1]
 [-1  1  1  1 -1  1  0  1 -1]
 [ 1  3  0  2  1  3  0  2 -1]
 [ 0  0  0  0  0  0  0  2  1]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]]
